In [1]:
import pickle
import os, glob
import rasterio

import xgboost as xgb
import numpy as np
import pandas as pd
from tqdm import tqdm
from rasterio.windows import Window

In [2]:
def predict_window(df_data, model):
    # print(df_data)
    df_data = xgb.DMatrix(df_data)
    X_predict = model.predict(df_data)
    return X_predict.astype('uint16')

In [3]:
def convert_img_to_input_model(array, list_number_band):
    i = 0
    dfObj = pd.DataFrame()
    for band in array:
        band = band.flatten()
        name_band = f"band {list_number_band[i]}"
        dfObj[name_band] = band
        i+=1
    return dfObj

## Predict 1 cửa sổ

In [4]:
def predict_one_window(array_window, model, list_number_band):
    shape_predict = array_window.shape[1:]
    df_window = convert_img_to_input_model(array_window, list_number_band)
    x_predict = predict_window(df_window, model)
    return np.array([x_predict.reshape(shape_predict)])

In [5]:
def write_window_many_chanel(output_ds, arr_c, s_h, e_h ,s_w, e_w, sw_w, sw_h, size_w_crop, size_h_crop):
    for c, arr in enumerate(arr_c):
        output_ds.write(arr[s_h:e_h,s_w:e_w],window = Window(sw_w, sw_h, size_w_crop, size_h_crop), indexes= c + 1)

In [6]:
def predict_big(out_fp_predict, fp_img_large, input_size, crop_size, model, list_number_band):
    with rasterio.open(fp_img_large) as src:
        h,w = src.height,src.width
        source_crs = src.crs
        source_transform = src.transform
        dtype_or = src.dtypes
        num_band = src.count
        
    with rasterio.open(out_fp_predict, 'w', driver='GTiff',
                                height = h, width = w,
                                count=1, dtype=dtype_or[0],
                                crs=source_crs,
                                transform=source_transform,
                                nodata=0,
                                compress='lzw') as output_ds:
        output_ds = np.empty((1,h,w))
        # output_ds.write_colormap(
        #             1, {
        #                 1: (255, 0, 0, 255),
        #                 2: (255,255,0, 255),
        #                 3:(128,0,0,255),
        #                 4:(0,255,0,255),
        #                 5:(0,128,0,255),
        #                 6:(0,0,255,255) })
        
        
    padding = int((input_size - crop_size)/2)
    list_weight = list(range(0, w, crop_size))
    list_hight = list(range(0, h, crop_size))


    src = rasterio.open(fp_img_large)

    with rasterio.open(out_fp_predict,"r+") as output_ds:
        with tqdm(total=len(list_hight)*len(list_weight)) as pbar:
            for start_h_org in list_hight:
                for start_w_org in list_weight:
                    # print('join')
                    # vi tri bat dau
                    h_crop_start = start_h_org - padding
                    w_crop_start = start_w_org - padding
                    # kich thuoc
                    tmp_img_size_model = np.zeros((num_band, input_size,input_size))
                    # truong hop 0 0
                    if h_crop_start < 0 and w_crop_start < 0:
                        # continue
                        h_crop_start = 0
                        w_crop_start = 0
                        size_h_crop = crop_size + padding
                        size_w_crop = crop_size + padding
                        img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                        tmp_img_size_model[:, padding:, padding:] = img_window_crop
                        img_predict = predict_one_window(tmp_img_size_model, model, list_number_band)
                        write_window_many_chanel(output_ds, img_predict, padding, crop_size + padding, padding, crop_size + padding, 
                                                                        start_w_org, start_h_org, crop_size, crop_size)
                    
                    # truong hop h = 0 va w != 0
                    elif h_crop_start < 0:
                        h_crop_start = 0
                        size_h_crop = crop_size + padding
                        size_w_crop = min(crop_size + 2*padding, w - start_w_org + padding)
                        img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                        
                        if size_w_crop == w - start_w_org + padding:
                            end_c_index_w =  size_w_crop
                            tmp_img_size_model[:,padding:,:end_c_index_w] = img_window_crop
                        else:
                            end_c_index_w = crop_size + padding
                            tmp_img_size_model[:, padding:,:] = img_window_crop
                        img_predict = predict_one_window(tmp_img_size_model, model, list_number_band)
                        write_window_many_chanel(output_ds, img_predict, padding, crop_size + padding ,padding, end_c_index_w, 
                                                    start_w_org, start_h_org,  min(crop_size, w - start_w_org), crop_size)
                    
                    # Truong hop w = 0, h!=0 
                    elif w_crop_start < 0:
                        w_crop_start = 0
                        size_w_crop = crop_size + padding
                        size_h_crop = min(crop_size + 2*padding, h - start_h_org + padding)
                        img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                        
                        if size_h_crop == h - start_h_org + padding:
                            end_c_index_h =  size_h_crop
                            tmp_img_size_model[:,:end_c_index_h,padding:] = img_window_crop
                        else:
                            end_c_index_h = crop_size + padding
                            tmp_img_size_model[:,:, padding:] = img_window_crop
                        img_predict = predict_one_window(tmp_img_size_model, model, list_number_band)
                        write_window_many_chanel(output_ds, img_predict, padding, end_c_index_h, padding, crop_size + padding, 
                                                    start_w_org, start_h_org, crop_size, min(crop_size, h - start_h_org))
                        
                    # Truong hop ca 2 deu khac khong
                    else:
                        size_w_crop = min(crop_size +2*padding, w - start_w_org + padding)
                        size_h_crop = min(crop_size +2*padding, h - start_h_org + padding)
                        img_window_crop  = src.read(window=Window(w_crop_start, h_crop_start, size_w_crop, size_h_crop))
                        # print(img_window_crop.shape, size_w_crop, size_h_crop)
                        if size_w_crop < (crop_size + 2*padding) and size_h_crop < (crop_size + 2*padding):
                            print(img_window_crop.shape, size_w_crop, size_h_crop)
                            end_c_index_h = size_h_crop
                            end_c_index_w = size_w_crop
                            tmp_img_size_model[:,:end_c_index_h,:   end_c_index_w] = img_window_crop
                        elif size_w_crop < (crop_size + 2*padding):
                            end_c_index_h = crop_size + padding
                            end_c_index_w = size_w_crop
                            tmp_img_size_model[:,:,:end_c_index_w] = img_window_crop
                        elif size_h_crop < (crop_size + 2*padding):
                            end_c_index_w = crop_size + padding
                            end_c_index_h = size_h_crop
                            tmp_img_size_model[:,:end_c_index_h,:] = img_window_crop
                        else:
                            end_c_index_w = crop_size + padding
                            end_c_index_h = crop_size + padding
                            tmp_img_size_model[:,:,:] = img_window_crop
                        img_predict = predict_one_window(tmp_img_size_model, model, list_number_band) 
                        write_window_many_chanel(output_ds, img_predict, padding, end_c_index_h, padding, end_c_index_w, 
                                                    start_w_org, start_h_org, min(crop_size, w - start_w_org), min(crop_size, h - start_h_org))
                    pbar.update()

## Predict one img

In [7]:
# out_fp_predict = r"E:\WORK\Cambodia_HatDieu\Data\img\Kep_xoaaaaaaaa_xgbost.tif"
# fp_img_large = r"E:\WORK\Cambodia_HatDieu\Data\img\Kep.tif"
# model_path = r"E:\WORK\Cambodia_HatDieu\Data\model_3000_v2.model"
# list_number_band = [1,2,3,4]
# input_size = 1000
# crop_size = 900
# model = xgb.Booster({'nthread': 4})
# model.load_model(model_path)
# print("done load")
# predict_big(out_fp_predict, fp_img_large, input_size, crop_size, model, list_number_band)

## Predict folder img

In [8]:
in_img_dir = r"E:\WORK\Mongodia\pixel_base\img"
# model_path = r"E:\WORK\Cambodia_HatDieu\Data\model_3000_v2_num_round_100.model"
model_path = r"E:\WORK\Mongodia\pixel_base\model_5000_v2_num_round_100_max_depth7_7Band.model"

out_dir_predict = os.path.join(in_img_dir + 'predict_xgboost' + os.path.basename(model_path)[:-6])
if not os.path.exists(out_dir_predict):
    os.makedirs(out_dir_predict)
print(out_dir_predict)

crop_size = 200
input_size = 300
list_number_band = [1,2,3,4,5,6,7]

model = xgb.Booster({'nthread': 20})
model.load_model(model_path)
model.set_param({"predictor": "gpu_predictor"})

list_fp_img = glob.glob(os.path.join(in_img_dir, '*.tif'))
for fp_img_large in list_fp_img:
    out_fp_predict = os.path.join(out_dir_predict, os.path.basename(fp_img_large))
    predict_big(out_fp_predict, fp_img_large, input_size, crop_size, model, list_number_band)
    with rasterio.open(out_fp_predict, 'w') as dst:
        



E:\WORK\Mongodia\pixel_base\imgpredict_xgboostmodel_5000_v2_num_round_100_max_depth7_7Band


  0%|          | 0/35 [00:00<?, ?it/s]

       band 1  band 2   band 3   band 4   band 5   band 6   band 7
0         0.0     0.0      0.0      0.0      0.0      0.0      0.0
1         0.0     0.0      0.0      0.0      0.0      0.0      0.0
2         0.0     0.0      0.0      0.0      0.0      0.0      0.0
3         0.0     0.0      0.0      0.0      0.0      0.0      0.0
4         0.0     0.0      0.0      0.0      0.0      0.0      0.0
...       ...     ...      ...      ...      ...      ...      ...
89995  9375.0  9732.0  11003.0  11316.0  17862.0  16612.0  13982.0
89996  8945.0  9346.0  10602.0  10717.0  18089.0  16380.0  13904.0
89997  8913.0  9441.0  10856.0  11233.0  18096.0  16427.0  13888.0
89998  9205.0  9719.0  11018.0  11598.0  16823.0  16470.0  14304.0
89999  9336.0  9774.0  10988.0  11242.0  18118.0  16255.0  13983.0

[90000 rows x 7 columns]


  9%|▊         | 3/35 [00:02<00:18,  1.70it/s]

       band 1  band 2   band 3   band 4   band 5   band 6   band 7
0         0.0     0.0      0.0      0.0      0.0      0.0      0.0
1         0.0     0.0      0.0      0.0      0.0      0.0      0.0
2         0.0     0.0      0.0      0.0      0.0      0.0      0.0
3         0.0     0.0      0.0      0.0      0.0      0.0      0.0
4         0.0     0.0      0.0      0.0      0.0      0.0      0.0
...       ...     ...      ...      ...      ...      ...      ...
89995  8554.0  8955.0  10273.0  10244.0  17376.0  16533.0  13387.0
89996  8595.0  9041.0  10376.0  10349.0  17190.0  16483.0  13369.0
89997  8302.0  8646.0   9875.0   9620.0  17544.0  15540.0  12323.0
89998  8284.0  8625.0   9795.0   9621.0  16491.0  15358.0  12312.0
89999  8317.0  8640.0   9690.0   9595.0  16007.0  15370.0  12391.0

[90000 rows x 7 columns]
       band 1  band 2   band 3   band 4   band 5   band 6   band 7
0         0.0     0.0      0.0      0.0      0.0      0.0      0.0
1         0.0     0.0      0.0      

...

 94%|█████████▍| 33/35 [00:05<00:00, 10.19it/s]

       band 1  band 2   band 3   band 4   band 5   band 6   band 7
0      8298.0  8912.0  10307.0  10815.0  16165.0  15753.0  13444.0
1      8048.0  8680.0  10213.0  10627.0  16028.0  15190.0  12929.0
2      8365.0  8877.0  10190.0  10679.0  15298.0  14525.0  12484.0
3      8597.0  9047.0  10206.0  10571.0  15679.0  14833.0  12492.0
4      8288.0  8504.0   9589.0   9427.0  16839.0  14550.0  11620.0
...       ...     ...      ...      ...      ...      ...      ...
89995     0.0     0.0      0.0      0.0      0.0      0.0      0.0
89996     0.0     0.0      0.0      0.0      0.0      0.0      0.0
89997     0.0     0.0      0.0      0.0      0.0      0.0      0.0
89998     0.0     0.0      0.0      0.0      0.0      0.0      0.0
89999     0.0     0.0      0.0      0.0      0.0      0.0      0.0

[90000 rows x 7 columns]
       band 1  band 2  band 3  band 4   band 5   band 6   band 7
0      8379.0  8518.0  9630.0  9323.0  17146.0  15088.0  11932.0
1      8211.0  8374.0  9416.0  9008.0  1

100%|██████████| 35/35 [00:05<00:00,  6.39it/s]

       band 1  band 2  band 3  band 4  band 5  band 6  band 7
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0
1         0.0     0.0     0.0     0.0     0.0     0.0     0.0
2         0.0     0.0     0.0     0.0     0.0     0.0     0.0
3         0.0     0.0     0.0     0.0     0.0     0.0     0.0
4         0.0     0.0     0.0     0.0     0.0     0.0     0.0
...       ...     ...     ...     ...     ...     ...     ...
89995     0.0     0.0     0.0     0.0     0.0     0.0     0.0
89996     0.0     0.0     0.0     0.0     0.0     0.0     0.0
89997     0.0     0.0     0.0     0.0     0.0     0.0     0.0
89998     0.0     0.0     0.0     0.0     0.0     0.0     0.0
89999     0.0     0.0     0.0     0.0     0.0     0.0     0.0

[90000 rows x 7 columns]


ValueError: 'driver' is required to write dataset.